In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import os
import numpy as np
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from skimage.transform import resize

from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras import Model
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2Config
import torchaudio
from torchaudio.transforms import Resample
import torch
import tensorflow_hub as hub
import kagglehub

c:\Users\kevin.KEVIN\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\kevin.KEVIN\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\kevin.KEVIN\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [13]:
train_data = pd.read_csv("combined_metadata_df.csv")
train_data.head()

,Unnamed: 0,Filename,Filepath,Gender,Emotion,Emotional Intensity
0,0,03-01-01-01-01-01-01.wav,Data/speech-emotion-recognition-en/Ravdess/aud...,male,neutral,medium
1,1,03-01-01-01-01-02-01.wav,Data/speech-emotion-recognition-en/Ravdess/aud...,male,neutral,medium
2,2,03-01-01-01-02-01-01.wav,Data/speech-emotion-recognition-en/Ravdess/aud...,male,neutral,medium
3,3,03-01-01-01-02-02-01.wav,Data/speech-emotion-recognition-en/Ravdess/aud...,male,neutral,medium
4,4,03-01-02-01-01-01-01.wav,Data/speech-emotion-recognition-en/Ravdess/aud...,male,neutral,medium


In [14]:
train_data = train_data[~(train_data['Emotion']=='surprised')]
train_data['Emotion'].value_counts()


Emotion
happy      1923
sad        1923
angry      1923
fear       1923
disgust    1923
neutral    1895
Name: count, dtype: int64

In [15]:
emotion_labels = train_data['Emotion'].unique().tolist()
emotion_keys = set(train_data.Emotion)
emotion_values = np.arange(0, len(emotion_keys))
emotion_dict = dict(zip(sorted(emotion_keys), emotion_values))
emotion_class=len(emotion_labels)

train_data['emotion_label'] = train_data['Emotion'].apply(lambda x: emotion_dict[x])
train_data.head()

,Unnamed: 0,Filename,Filepath,Gender,Emotion,Emotional Intensity,emotion_label
0,0,03-01-01-01-01-01-01.wav,Data/speech-emotion-recognition-en/Ravdess/aud...,male,neutral,medium,4
1,1,03-01-01-01-01-02-01.wav,Data/speech-emotion-recognition-en/Ravdess/aud...,male,neutral,medium,4
2,2,03-01-01-01-02-01-01.wav,Data/speech-emotion-recognition-en/Ravdess/aud...,male,neutral,medium,4
3,3,03-01-01-01-02-02-01.wav,Data/speech-emotion-recognition-en/Ravdess/aud...,male,neutral,medium,4
4,4,03-01-02-01-01-01-01.wav,Data/speech-emotion-recognition-en/Ravdess/aud...,male,neutral,medium,4


In [16]:
yamnet_model = hub.load('https://www.kaggle.com/models/google/yamnet/TensorFlow2/yamnet/1')

In [17]:
import torchaudio
import torch
import numpy as np
from sklearn.decomposition import PCA

def audio_processing(file_path, target_duration=3, target_sample_rate=16000, device=None, target_embedding_dim = 256):
    # Set the device (CPU or CUDA)
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the audio file
    waveform, sample_rate = torchaudio.load(file_path)
    waveform = waveform.to(device)  # Move waveform to the device

    # Convert to mono if stereo
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Resample if the sample rate is different from the target
    if sample_rate != target_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate).to(device)
        waveform = resampler(waveform)
        sample_rate = target_sample_rate

    # Ensure the audio is of target duration
    target_length = target_sample_rate * target_duration
    num_samples = waveform.shape[1]

    if num_samples < target_length:
        # Pad with zeros at the end
        padding = target_length - num_samples
        waveform = torch.nn.functional.pad(waveform, (0, padding), mode="constant", value=0)
    elif num_samples > target_length:
        # Truncate to the target length
        waveform = waveform[:, :target_length]

    # Process with YAMNet model (requires NumPy for YAMNet)
    waveform_np = waveform.squeeze().cpu().numpy()  # Convert to NumPy for compatibility with YAMNet
    scores, embeddings, log_mel_spectrogram = yamnet_model(waveform_np)

    # Check output shapes
    assert scores.shape[1] == 521, "Scores output shape mismatch."
    assert embeddings.shape[1] == 1024, "Embeddings output shape mismatch."
    assert log_mel_spectrogram.shape[1] == 64, "Log Mel Spectrogram shape mismatch."

    return scores, embeddings, log_mel_spectrogram

In [25]:
import numpy as np
from tqdm import tqdm

# Initialize lists
X_train_embeddings = []
X_train_spectrograms = []
y_train = [] 
labels = []

# Define the number of emotion classes
emotion_class = 6  # Replace with your actual number of classes

# Iterate over the training data
for index, file in tqdm(enumerate(train_data['Filepath']), total=len(train_data['Filepath']), desc="Processing audio files"):
    _, embedding, log_mel_spectrogram = audio_processing(file)  # Ensure this function is defined
    X_train_embeddings.append(embedding)
    X_train_spectrograms.append(log_mel_spectrogram)
    
    # Get the label using integer-based indexing
    label = train_data['emotion_label'].iloc[index]
    
    # Ensure the label is an integer
    label = int(label)
    
    # Create one-hot encoded label
    temp = np.zeros(emotion_class)
    temp[int(label)] = 1
    y_train.append(temp)
    
    labels.append(label)


Processing audio files: 100%|██████████| 11510/11510 [41:33<00:00,  4.62it/s]


In [27]:
np.save("X_train_embeddings.npy", X_train_embeddings)
np.save("X_train_spectrograms.npy", X_train_spectrograms)
np.save("y_train.npy", y_train)
np.save("labels.npy", labels)